# Домашнее задание
1. Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

Каскадные метрики:


- Expected reciprocal rank (ERR) — метрика качества ранжирования, основанная на каскадной модели.
- PFound — метрика качества ранжирования, использующая похожую на каскадную модель


2. Доделать все функции, где стоит комментарий "сделать дома"

## 1. hit rate at K

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [11]:
import pandas as pd
import numpy as np

recommended_list = [27, 156, 1134, 991, 143, 1543, 3345, 533, 11, 43] #id товаров
prices_recommended = [400, 60, 40, 40 , 90, 80, 70, 70, 90, 120]

bought_list = [521, 32, 143, 991]
prices_bought = [101,110,90,150]

In [12]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

print(f"hit_rate = {hit_rate(recommended_list, bought_list)}\n")
print(f"hit_rate_at_k = {hit_rate_at_k(recommended_list, bought_list, 4)}\n")

hit_rate = 1

hit_rate_at_k = 1



## 2. Precision at k

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

In [13]:


def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

print(f"precision = {precision(recommended_list, bought_list)}\n")
print(f"precision_at_k = {precision_at_k(recommended_list, bought_list, 5)}\n")
print(f"money_precision_at_k = {money_precision_at_k_(recommended_list, bought_list, prices_recommended, 5)}\n")

precision = 0.2

precision_at_k = 0.4

money_precision_at_k = 0.20634920634920634



## 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [14]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommend_list, bought_list)
    
    recall = np.dot(flags, prices_recommended).sum() / prices_bought.sum()
    
    return recall

print(f"recall = {recall(recommended_list, bought_list)}\n")
print(f"recall_at_k = {recall_at_k(recommended_list, bought_list, 5)}\n")
print(f"money_recall_at_k = {money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 5)}\n")

recall = 0.5

recall_at_k = 0.5

money_recall_at_k = 0.28824833702882485



## 4 MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [15]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant


def map_k(recommended_list_users, bought_list_users, k=5):
    user_qty = len(recommended_list_users)
    ap_k_sum = sum([ap_k(recommended_list_users[i], bought_list_users[i], k) for i in range(0,user_qty)])
    return (1/user_qty)*ap_k_sum

print(map_k(recommended_list_3_users, bought_list_3_users, 5))
    

0.3333333333333333


## 5 Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [16]:
def dcg_at_k(r, k, method=0):
    
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(recommended_list, bought_list, k=5, method=0):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    r = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(r) == 0:
        return 0
    
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

print(ndcg_at_k(recommended_list, bought_list))

1.0


## Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [17]:
def mean_reciprocal_rank_at_k(recommended_list_users, bought_list_users, k=5): 
    rs = []
    users_qty = len(recommended_list_users)
    
    i = 0
    while i < users_qty:
        bought_list = np.array(bought_list_users[i])
        recommended_list = np.array(recommended_list_users[i])[:k]
        
        rs.append(np.nonzero(np.isin(recommended_list, bought_list))[0])
        i = i+1
        
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

print(mean_reciprocal_rank_at_k(recommended_list_3_users, bought_list_3_users))

0.3333333333333333
